In [1]:
from imprint.nb_util import setup_nb
setup_nb()
import asyncio
import redis
import numpy as np

import confirm.cloud.clickhouse as ch

from confirm.adagrid.validate import AdaValidate
from confirm.adagrid.calibrate import AdaCalibrate
from confirm.adagrid.validate import ada_validate
from confirm.adagrid.calibrate import ada_calibrate
from confirm.cloud.redis_heartbeat import HeartbeatThread
import confirm.adagrid.adagrid as adagrid
import imprint as ip
from imprint.models.ztest import ZTest1D

### Glossary

- **coordination**: All workers come together and divide up all tiles.
- **step**: A worker checks the convergence criterion, select tiles, and simulates.
- **packet**: The unit of simulation work that a worker requests from the DB.
- **batch**: The unit of simulation work that is passed to a Model.

In [3]:
db = ch.Clickhouse.connect(job_id = "ab7fd2cf834a4c6795a388201a8715d8")

INFO:confirm.cloud.clickhouse:[worker_id=None] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/None
INFO:confirm.cloud.clickhouse:[worker_id=None] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/ab7fd2cf834a4c6795a388201a8715d8
INFO:confirm.cloud.clickhouse:[worker_id=None] 
Connected to job ab7fd2cf834a4c6795a388201a8715d8


In [4]:
df = db.get_results()

In [9]:
ch._query_df(db.client, "select * from results")

,id,active,parent_id,theta0,radii0,null_truth0,coordination_id,step_id,worker_id,packet_id,creator_id,creation_time,processor_id,processing_time,eligible,grid_cost,impossible,orderer,K,alpha0,idx,twb_max_lams,twb_mean_lams,twb_min_lams,lams,B_lams0,B_lams1,B_lams2,B_lams3,B_lams4,twb_lams0,twb_lams1,twb_lams2,twb_lams3,twb_lams4
0,137439477761,0,262144,-0.25,0.25,1,0,1,2,0,2,3,2,3,0,0.012713,0,-2.096647,8192,0.012287,99,-1.982016,-2.012509,-2.096647,-1.990893,-2.019755,-2.004284,-2.006723,-1.993602,-1.948296,-1.986463,-1.990693,-2.096647,-2.006723,-1.982016
1,137439477760,0,262144,-0.75,0.25,1,0,1,2,0,2,3,2,3,0,0.012713,0,-1.596647,8192,0.012287,99,-1.482016,-1.512509,-1.596647,-1.490893,-1.519755,-1.504284,-1.506723,-1.493602,-1.448296,-1.486463,-1.490693,-1.596647,-1.506723,-1.482016
2,262144,0,0,-0.50,0.50,1,0,0,2,0,2,1,2,1,0,0.019327,0,-2.111800,8192,0.005673,45,-1.953229,-2.032202,-2.111800,-2.042335,-2.054449,-2.100243,-2.065204,-1.995995,-1.988442,-2.065204,-1.988442,-2.111800,-2.042335,-1.953229


In [7]:
ch._query_df(db.client, "select * from done")

,coordination_id,worker_id,step_id,packet_id,id,active,finisher_id,refine,deepen,split
0,0,0,0,0,0,0,0,0,0,0
1,0,2,0,0,262144,0,2,1,0,0
2,0,2,1,0,137439477760,0,2,0,1,0
3,0,2,1,0,137439477761,0,2,1,0,0


In [8]:
ch._query_df(db.client, "select * from tiles")

,id,active,parent_id,theta0,radii0,null_truth0,K,coordination_id,step_id,worker_id,packet_id,creator_id,creation_time
0,262144,1,0,-0.500,0.500,1,8192,0,0,2,0,2,1
1,137439477760,1,262144,-0.750,0.250,1,8192,0,1,2,0,2,3
2,137439477761,1,262144,-0.250,0.250,1,8192,0,1,2,0,2,3
3,206158954496,1,137439477760,-0.750,0.250,1,16384,0,2,2,0,2,5
4,274878431232,1,137439477761,-0.375,0.125,1,8192,0,2,2,0,2,5
5,274878431233,1,137439477761,-0.125,0.125,1,8192,0,2,2,0,2,5


In [6]:
df

,id,parent_id,theta0,radii0,null_truth0,coordination_id,step_id,worker_id,packet_id,creator_id,creation_time,processor_id,processing_time,grid_cost,impossible,orderer,K,alpha0,idx,twb_max_lams,twb_mean_lams,twb_min_lams,lams,B_lams0,B_lams1,B_lams2,B_lams3,B_lams4,twb_lams0,twb_lams1,twb_lams2,twb_lams3,twb_lams4,active,eligible
0,262144,0,-0.50,0.50,1,0,0,2,0,2,1,2,1,0.019327,0,-2.111800,8192,0.005673,45,-1.953229,-2.032202,-2.111800,-2.042335,-2.054449,-2.100243,-2.065204,-1.995995,-1.988442,-2.065204,-1.988442,-2.111800,-2.042335,-1.953229,False,False
1,137439477761,262144,-0.25,0.25,1,0,1,2,0,2,3,2,3,0.012713,0,-2.096647,8192,0.012287,99,-1.982016,-2.012509,-2.096647,-1.990893,-2.019755,-2.004284,-2.006723,-1.993602,-1.948296,-1.986463,-1.990693,-2.096647,-2.006723,-1.982016,False,False
2,137439477760,262144,-0.75,0.25,1,0,1,2,0,2,3,2,3,0.012713,0,-1.596647,8192,0.012287,99,-1.482016,-1.512509,-1.596647,-1.490893,-1.519755,-1.504284,-1.506723,-1.493602,-1.448296,-1.486463,-1.490693,-1.596647,-1.506723,-1.482016,False,False


In [4]:
get_ipython().__class__.__name__

'ZMQInteractiveShell'

In [5]:
get_ipython().config

{'IPKernelApp': {'connection_file': '/Users/tbent/Library/Jupyter/runtime/kernel-1499f3c3-e1c4-42f7-a40d-1ecc290dfaa8.json'},
 'Completer': {'use_jedi': False},
 'InlineBackend': {'figure_format': 'retina'}}

In [6]:
asyncio.get_running_loop()

<_UnixSelectorEventLoop running=True closed=False debug=False>

In [2]:
ch.clear_dbs(drop_all_redis_keys=True)

INFO:confirm.cloud.clickhouse:[worker_id=None] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/None


Dropping the following databases:
['8d239a4d499b4431b0cd2b01f367a68d']
Are you sure? [yN]
drop database 8d239a4d499b4431b0cd2b01f367a68d
deleting redis keys [b'8d239a4d499b4431b0cd2b01f367a68d:coordination_id', b'8d239a4d499b4431b0cd2b01f367a68d:next_worker_id', b'8d239a4d499b4431b0cd2b01f367a68d:worker_1:step_0:n_packets', b'8d239a4d499b4431b0cd2b01f367a68d:worker_1:step_0:n_tiles', b'8d239a4d499b4431b0cd2b01f367a68d:worker_2:step_0:n_packets', b'8d239a4d499b4431b0cd2b01f367a68d:worker_2:step_0:n_tiles', b'8d239a4d499b4431b0cd2b01f367a68d:coordination_id', b'8d239a4d499b4431b0cd2b01f367a68d:next_worker_id', b'8d239a4d499b4431b0cd2b01f367a68d:worker_1:step_0:n_packets', b'8d239a4d499b4431b0cd2b01f367a68d:worker_1:step_0:n_tiles', b'8d239a4d499b4431b0cd2b01f367a68d:worker_2:step_0:n_packets', b'8d239a4d499b4431b0cd2b01f367a68d:worker_2:step_0:n_tiles']
drop_all_redis_keys=True, deleting redis keys []


In [3]:
db = ch.Clickhouse.connect()
g = ip.cartesian_grid(
    theta_min=[-1], theta_max=[1], n=[50], null_hypos=[ip.hypo("x0 < 0")]
)
ada_validate(ZTest1D, lam=-1.96, g=g, db=db, prod=False)

INFO:confirm.cloud.clickhouse:[worker_id=None] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/None
INFO:confirm.cloud.clickhouse:[worker_id=None] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/627e5213900d41a985f3d73930cd19db
INFO:confirm.cloud.clickhouse:[worker_id=None] 
Connected to job 627e5213900d41a985f3d73930cd19db
DEBUG:imprint.grid:[worker_id=None] 
_gen_short_uuids(n=50, worker_id=1, t=1676415808, n_bits=18, worker_bits=18) = [4521952675461595136 4521952675461595137 4521952675461595138, ...]:


RuntimeError: Cannot run the event loop while another loop is running

In [5]:
ada.db.get_tiles()

,id,parent_id,theta0,radii0,null_truth0,K,coordination_id,step_id,worker_id,packet_id,creator_id,creation_time,active
0,4521935976628748288,0,-0.99998,0.00002,1,8192,0,0,1,0,1,1.676416e+09,1
1,4521935976628748289,0,-0.99994,0.00002,1,8192,0,0,1,0,1,1.676416e+09,1
2,4521935976628748290,0,-0.99990,0.00002,1,8192,0,0,1,0,1,1.676416e+09,1
3,4521935976628748291,0,-0.99986,0.00002,1,8192,0,0,1,0,1,1.676416e+09,1
4,4521935976628748292,0,-0.99982,0.00002,1,8192,0,0,1,0,1,1.676416e+09,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,4521935976628773283,0,-0.00018,0.00002,1,8192,0,0,2,12,1,1.676416e+09,1
24996,4521935976628773284,0,-0.00014,0.00002,1,8192,0,0,2,12,1,1.676416e+09,1
24997,4521935976628773285,0,-0.00010,0.00002,1,8192,0,0,2,12,1,1.676416e+09,1
24998,4521935976628773286,0,-0.00006,0.00002,1,8192,0,0,2,12,1,1.676416e+09,1


In [12]:
async with ada.db.heartbeat(ada.worker_id):
    await ada._run_local()

DEBUG:confirm.cloud.redis_heartbeat:[worker_id=1] 
Acquired lock 793bd9f8305b493b9cda375cc31ac8fb:heartbeat:1
DEBUG:confirm.cloud.redis_heartbeat:[worker_id=1] 
Added worker 1 to workers set.
DEBUG:confirm.cloud.redis_heartbeat:[worker_id=1] 
Extended lock 793bd9f8305b493b9cda375cc31ac8fb:heartbeat:1 for 30 seconds.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Skipping packet. Flag 793bd9f8305b493b9cda375cc31ac8fb:worker_1:step_4:packet_0:insert is set by worker_id=1.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Empty packet indicates that we might be done with work for this step.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Done with work for this step.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Convergence!!
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Waiting for coordination 1
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Leading the coordination
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Waiting workers: [1]
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Registered workers: [1]
D

In [8]:
async with HeartbeatThread(ada.db.redis_con, ada.db.job_id, ada.worker_id):
    await ada.process_step(0, 0)

DEBUG:confirm.adagrid:[worker_id=None] 
Acquired lock dcfe4aff65e84386af721900203e0317:heartbeat:1
DEBUG:confirm.adagrid:[worker_id=None] 
Added worker 1 to workers set.
DEBUG:confirm.adagrid:[worker_id=None] 
Extended lock dcfe4aff65e84386af721900203e0317:heartbeat:1 for 30 seconds.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Skipping packet. Flag dcfe4aff65e84386af721900203e0317:worker_1:step_0:packet_0:insert is set by worker_id=1.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Skipping packet. Flag dcfe4aff65e84386af721900203e0317:worker_1:step_0:packet_1:insert is set by worker_id=1.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Skipping packet. Flag dcfe4aff65e84386af721900203e0317:worker_1:step_0:packet_2:insert is set by worker_id=1.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Skipping packet. Flag dcfe4aff65e84386af721900203e0317:worker_1:step_0:packet_3:insert is set by worker_id=1.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Skipping packet. Flag dcfe4aff65e84

In [ ]:
db.get_reports()

,worker_id,step_id,packet_id,n_tiles,runtime_get_work,runtime_process_tiles,runtime_insert_results,status
4,1,0,0,1024,0.089789,0.064014,0.000085,WORKING
3,1,0,1,1024,0.086273,0.018211,0.000090,WORKING
0,1,0,2,1024,0.114576,0.013122,0.000064,WORKING
5,1,0,3,1024,0.026534,0.014037,0.000126,WORKING
2,1,0,4,1024,0.046059,0.012358,0.000127,WORKING
1,1,0,5,1024,0.177392,0.023063,0.000092,WORKING
6,1,0,6,1024,0.041068,0.015855,0.000069,WORKING
8,1,0,7,1024,0.050865,0.014689,0.000110,WORKING
9,1,0,8,1024,0.132194,0.016320,0.000176,WORKING
7,1,0,9,1024,0.033667,0.016473,0.000112,WORKING


In [ ]:
await ada.new_step(1)

INFO:confirm.adagrid.validate:[worker_id=None] 
Preparing new step with 1024 parent tiles.
DEBUG:confirm.cloud.clickhouse:[worker_id=None] 
finish:                     id  step_id  packet_id  active  finisher_id  refine  \
0  4515240363132477857        0         24   False            1   False   
1  4515240363132477858        0         24   False            1   False   
2  4515240363132477859        0         24   False            1   False   
3  4515240363132477860        0         24   False            1   False   
4  4515240363132477861        0         24   False            1   False   

   deepen  split  
0    True  False  
1    True  False  
2    True  False  
3    True  False  
4    True  False  
DEBUG:imprint.grid:[worker_id=None] 
_gen_short_uuids(n=1024, worker_id=1, t=1676318137, n_bits=18, worker_bits=18) = [4515240775449313280 4515240775449313281 4515240775449313282, ...]:
DEBUG:imprint.grid:[worker_id=None] 
_gen_short_uuids(n=0, worker_id=1, t=1676318138, n_bits=18, work

<WorkerStatus.NEW_STEP: 5>

In [11]:
# TODO: test running coordinate with a single work and confirming that it
# updates eligible and active.
pre = ch._query_df(db.client, 'select * from results')
pre['active'].sum(), pre['eligible'].sum()

(25000, 25000)

In [25]:
async with HeartbeatThread(ada.db.redis_con, ada.db.job_id, ada.worker_id):
    status = await ada.coordinate()

In [20]:
ch._query_df(db.client, 'select * from results')['active'].sum()

25000

In [23]:
ch._query_df(db.client, 'select * from results where coordination_id = 1')

,id,active,parent_id,theta0,radii0,null_truth0,step_id,creator_id,creation_time,coordination_id,worker_id,packet_id,processor_id,processing_time,eligible,grid_cost,sim_cost,total_cost,total_cost_order,tie_bound_order,tie_sum,tie_est,tie_cp_bound,tie_bound,K


In [12]:
ada.db.get_tiles()

,id,parent_id,theta0,radii0,null_truth0,K,step_id,creator_id,creation_time,worker_id,packet_id,active
0,4514895185200807936,0,-0.99998,0.00002,1,8192,0,1,1.676313e+09,1,0,1
1,4514895185200807937,0,-0.99994,0.00002,1,8192,0,1,1.676313e+09,1,0,1
2,4514895185200807938,0,-0.99990,0.00002,1,8192,0,1,1.676313e+09,1,0,1
3,4514895185200807939,0,-0.99986,0.00002,1,8192,0,1,1.676313e+09,1,0,1
4,4514895185200807940,0,-0.99982,0.00002,1,8192,0,1,1.676313e+09,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
26019,4514895597517669371,4514895185200831859,-0.04306,0.00002,1,16384,1,1,1.676313e+09,1,0,1
26020,4514895597517669372,4514895185200831860,-0.04302,0.00002,1,16384,1,1,1.676313e+09,1,0,1
26021,4514895597517669373,4514895185200831861,-0.04298,0.00002,1,16384,1,1,1.676313e+09,1,0,1
26022,4514895597517669374,4514895185200831862,-0.04294,0.00002,1,16384,1,1,1.676313e+09,1,0,1


In [10]:
await ada.process_step(1)

DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
Simulated 1024 tiles in 0.07 seconds.
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
No more work for this step.
DEBUG:confirm.cloud.clickhouse:[worker_id=None] 
writing 1024 results
DEBUG:confirm.adagrid.adagrid:[worker_id=None] 
inserted packet results for (step_id = 1, packet_id=0) with 1024 results
